# causalgraph Basics

This tutorial describes the basic architecture and functionalities of the causalgraph module.

Required packages to run this tutorial are:
- jupyter (to run this notebook)
- [owlready2 0.35](https://owlready2.readthedocs.io/en/v0.35/) (backend for the causalgraph store)

#### Beneficial prior knowledge
causalgraph (cg) relies on semantic technology, mainly through the python package 'owlready2'.
The core functions are already implemented in owlready2 and wrapped in cg to avoid misuse. Therefore, the author advises to look at the tool [owlready2](https://owlready2.readthedocs.io/en/v0.35/) and get to know some of its essential functionality. Especially the tutorial on [creating individuals](https://owlready2.readthedocs.io/en/v0.35/class.html) is an excellent basic knowledge to get to know the inner workings of cg.

### Understanding the available classes in the ontology

The ontology describes the classes and the allowed relations between its instances (called 'individuals' in semantic web speech).  
The **'causalgraph-ontology'** is maintained [as part of the causalgraph group as well](https://github.com/causalgraph/causalgraph-ontology)

To understand the available classes and relations, it's best to look at the documentation of the ontology found here: **https://causalgraph.github.io/causalgraph-ontology/**.


#### CausalNodes and CausalEdges
The most important concept for this tutorial is the relation between [*CausalNodes*](https://causalgraph.github.io/causalgraph-ontology/#CausalNode) and [*CausalEdges*](https://causalgraph.github.io/causalgraph-ontology/#CausalEdge). 

##### CausalNodes
A CausalNode can either represent an effect, a cause or both (e.g. effect of Node1 and cause for Node3).  
CausalNodes are not causally connected by direct link, but through an Individual of Type *CausalEdge*:

![](https://mermaid.ink/svg/eyJjb2RlIjoiZ3JhcGggVERcbiAgICBzdWJncmFwaCBDYXVzYWwgU3RydWN0dXJlXG4gICAgICAgIEVkZ2V7Q2F1c2FsRWRnZX0gLS0-IHxoYXNfY2F1c2V8IE5vZGUxKENhdXNhbE5vZGUpXG4gICAgICAgIEVkZ2UgLS0-IHxoYXNfZWZmZWN0fCBOb2RlMihDYXVzYWxOb2RlKVxuICAgIGVuZCBcblxuICAgICIsIm1lcm1haWQiOnsidGhlbWUiOiJkZWZhdWx0In0sInVwZGF0ZUVkaXRvciI6ZmFsc2UsImF1dG9TeW5jIjp0cnVlLCJ1cGRhdGVEaWFncmFtIjpmYWxzZX0)

##### CausalEdges
The CausalEdge is introduced to hold further information about the causal connection. E.g.:
- *Confidence* (in it's existence), 
- *Creator* (Human Expert or ML-Algorithm), 
- *TimeLag* between Cause and Effect,
- ... 

This is not modeled in *causalgraph-ontology* yet, but can be imagined similarly to this:

![](https://mermaid.ink/svg/eyJjb2RlIjoiZ3JhcGggVERcbiAgICBzdWJncmFwaCBDYXVzYWwgU3RydWN0dXJlXG4gICAgICAgIEVkZ2V7Q2F1c2FsRWRnZX0gLS0-IHxoYXNfY2F1c2V8IE5vZGUxKENhdXNhbE5vZGUpXG4gICAgICAgIEVkZ2UgLS0-IHxoYXNfZWZmZWN0fCBOb2RlMihDYXVzYWxOb2RlKVxuICAgIGVuZCBcbiAgICBzdWJncmFwaCBBZGRpdG9uYWwgQ2F1c2FsIEVkZ2UgSW5mb3JtYXRpb25cbiAgICAgICAgRWRnZSAtLT4gfGhhc19jcmVhdG9yfCBDcmVhdG9yWy4uLkNyZWF0b3JdXG4gICAgICAgIEVkZ2UgLS0-IHxoYXNfdGltZV9sYWd8IFRpbWVsYWdbLi4uVGltZWxhZ11cbiAgICBlbmRcbiAgICAiLCJtZXJtYWlkIjp7InRoZW1lIjoiZGVmYXVsdCJ9LCJ1cGRhdGVFZGl0b3IiOmZhbHNlLCJhdXRvU3luYyI6dHJ1ZSwidXBkYXRlRGlhZ3JhbSI6ZmFsc2V9)


## Working with the 'cg' module

### Loading the module
The facade class for the module is 'Graph' which is loaded below.

In [ ]:
# load the requirements and the module
# add the current path to the sys.path to find 'causalgraph.Graph'
import os
import sys
sys.path.insert(1, os.path.join(sys.path[0], '..'))
# general imports
import time
# causalgraph imports
import causalgraph.utils.owlready2_utils as owlutils
from causalgraph.utils.path_utils import get_project_root
print(get_project_root())

In [ ]:
# load the 'Graph' module
from causalgraph import Graph

### Instantiate the 'Graph'

During instantiation, the '**Graph**' loads the class-structure-defining ontology ('onto' in the examples at [owlready2-docs](https://owlready2.readthedocs.io/en/v0.35/onto.html)) into **Graph.store**.

This ontology and the individuals created from it are saved in sqlite3 quadstore, which is linked by the attribute **Graph.world**. To read more about an owlready2.World please refer to ['owlready2.World'](https://owlready2.readthedocs.io/en/v0.35/world.html).

In [ ]:
# specify new sqlite db for this example
# delete old SQL-DB if exists (restart of kernel may be necessary)
sql_file_name='example1.sqlite3'
if os.path.exists(sql_file_name):
    os.remove(sql_file_name)
    print(f"Deleted old db with name {sql_file_name}")
# Init Graph
G = Graph(sql_db_filename=sql_file_name)

#### Show that the classes are loaded and no individuals exist

The Ontology is accessable with *Graph.store*. To show its available classes, properties and individuals (none initially) we do the following:

In [ ]:
# available classes and properties:
print(f"Available classes:\n {list(G.store.classes())}\n")
print(f"Available properties:\n {list(G.store.properties())}\n")
# at startup no individuals are available:
print(f"Available individuals:\n {list(G.store.individuals())}")

#### Add CausalNodes and CausalEdges

All the 'Add' functions are grouped in the Object *Graph.add*. Therefore one can add CausalEdges and CausalNodes in the following way:

In [ ]:
# add two CausalNodes and show these individuals:
G.add.causal_node("cause")
G.add.causal_node("effect")
print(f"Now we have these individuals:\n {list(G.store.individuals())}")

In [ ]:
# Add a CausalEdge between those two nodes:
G.add.causal_edge("cause", "effect", "causal_edge")
print(f"Now we have these individuals:\n {list(G.store.individuals())}")

In [ ]:
# Add a CausalEdge between 'effect' and 'another_cause'
G.add.causal_node("another_cause")
G.add.causal_edge('another_cause', 'effect', "another_edge")
print(f"Not we have these individuals:\n {list(G.store.individuals())}")

#### Show the properties of 'cg_store.effect' and 'cg_store.causal_edge'
With functions from owlready2, one can visualize the properties of individuals. We will do this for the CausalNode ```cg_store.effect``` which has the incoming edge ```cg_store_causal_edge``` and the outgoing edge ```cg_store.another_edge```. Printing the properties of CausalEdge ```cg_store.causal_edge``` shows that it connects the nodes ```cg_store.effect``` and ```cg_store.cause```.

In [ ]:
# Inspect 'cg_store.effect'
inspected_node = owlutils.get_entity_by_name('effect', G.store)
for prop in inspected_node.get_properties():
    for value in prop[inspected_node]:
        print(".%s == %s" % (prop.python_name, value))

In [ ]:
# Inspect 'cg_store.causal_edge'
inspected_node = owlutils.get_entity_by_name('causal_edge', G.store)
for prop in inspected_node.get_properties():
    for value in prop[inspected_node]:
        print(".%s == %s" % (prop.python_name, value))

In [ ]:
# Visualize resulting graph
G.draw.nx()
#G.draw.html(directory='.', filename='test')

##### Remove CausalNodes and CausalEdges

The ```Remove``` functions are grouped in the Object ```Graph.remove```. Therefore one can remove CausalEdges and CausalNodes in the following way:

In [ ]:
# Remove two CausalNodes and show the rest of individuals:
print("Graph before remove")
G.draw.nx()
G.remove.causal_node("another_cause",)
print("Graph after remove")
G.draw.nx()
print(f"Now we have these individuals:\n {list(G.store.individuals())}")

remove causal edges 

In [ ]:
# Remove one edge and show the rest of individuals:
print("Graph before remove")
G.draw.nx()
G.remove.causal_edge_by_name("causal_edge")
print("Graph after remove")
G.draw.nx()
print(f"Now we have these individuals:\n {list(G.store.individuals())}")

In [ ]:
# Remove one edge, between two nodes and show the rest of individuals:
## adds two edges between 'cause' and 'effect'
G.add.causal_node("another_effect")
G.add.causal_edge("cause", "effect", "edge_1")
G.add.causal_edge("effect", "cause", "edge_2")
G.add.causal_edge("cause", "another_effect", "edge_3")
G.add.causal_edge("effect", "another_effect", "edge_4")
print("Graph before remove")
G.draw.nx()
print(f"Now we have these individuals:\n {list(G.store.individuals())}")
## deletes all edges between 'cause' and 'effect'.
G.remove.causal_edges("cause", "effect")
print("Graph after remove")
G.draw.nx()
print(f"Now we have these individuals:\n {list(G.store.individuals())}")

remove all causalEdges which are connected with one causalNode

In [ ]:
# Remove one edge, between two nodes and show the rest of individuals:
## adds two edges between 'cause' and 'effect'
G.add.causal_node("another_cause")
G.add.causal_edge("cause", "effect", "edge_5")
G.add.causal_edge("effect", "cause", "edge_6")
G.add.causal_edge("another_effect", "another_cause", "edge_7")
print("Graph before remove")
G.draw.nx()
print(f"Now we have these individuals:\n {list(G.store.individuals())}")
## deletes all edges that are connected with 'effect 
G.remove.causal_edges_from_node("effect")
print("Graph after remove")
G.draw.nx()
print(f"Now we have these individuals:\n {list(G.store.individuals())}")